In [4]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
from TravelDashboard.data import preproc_flight_data, get_data, save_processed_data

In [99]:
# get path of all .csv files (history from API crawling)
all_files = glob.glob(os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath('')))),"jonas_pieper","project","api_call","output", "*.csv"))

In [100]:
txt = all_files

In [101]:
test = [int(word[-14:-4]) for word in txt]

In [107]:
[i for i, x in enumerate(test) if x>1652969272]

[0,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 1

In [102]:
max(test)

1654705311

In [88]:
myfiles = glob.glob(os.path.join(os.path.dirname(os.path.abspath('')),"raw_data", "preproc_data_test", "*.csv"))

In [93]:
test = [word[-14:-4] for word in txt]
max(test)

'1654700158'

In [15]:
print("Load local databases ...")
# Load required databases for matching/ preprocessing
seat_data = pd.read_csv(os.path.join(os.path.dirname(os.path.abspath('')),"TravelDashboard", "data","seats_p_aircraft_final.csv"),
                        index_col=0)

#seat_data = pd.read_csv("TravelDashboard/data/seats_p_aircraft_final.csv",
#                        index_col=0)
aircraftregister_US = pd.read_csv(os.path.join(
    os.path.dirname(os.path.abspath('')), "TravelDashboard", "data",
    "US_ReleasableAircraftRegister.csv"),
                                    index_col=0,
                                    low_memory=False)
opensky_DB = pd.read_csv(os.path.join(
    os.path.dirname(os.path.abspath('')), "TravelDashboard", "data",
    "OpenSky_AircraftDatabase.csv"))


Load local databases ...


In [98]:
# here I will have to implement the foor loop 
for i, file in enumerate(all_files[350:1000]):
    print(i+350)
    # Define column names for dataframe
    col_names = [
        'icao24', 'callsign', 'origin_country', 'time_position',
        'last_contact', 'long', 'lat', 'baro_altitude', 'on_ground',
        'velocity', 'true_track', 'vertical_rate', 'sensors', 'geo_altitude',
        'squawk', 'spi', 'position_source', "drop", "time"
    ]

    #Load raw data
    print("Load raw .csv files ...")
    # Load dataframe
    flights = pd.read_csv(file, index_col=0, header=0)

    if flights.shape[1]==19:
        flights.columns=col_names
        flights.drop(columns=[
        "time_position", "last_contact", "sensors", "squawk", "spi",
        "position_source", "drop"], inplace=True)  # drop columns
    else: 
        flights.columns=col_names = [
        'icao24', 'callsign', 'origin_country', 'time_position',
        'last_contact', 'long', 'lat', 'baro_altitude', 'on_ground',
        'velocity', 'true_track', 'vertical_rate', 'sensors', 'geo_altitude',
        'squawk', 'spi', 'position_source', "time"]
        flights.drop(columns=[
        "time_position", "last_contact", "sensors", "squawk", "spi",
        "position_source"], inplace=True)  # drop columns

    flights = flights.dropna(
        subset=['lat', "long", "icao24"])  #drop rows where lat or long or icao24 is NaN -> can not be used for further processing

    pflights_df_final = preproc_flight_data(flights, opensky_DB, seat_data, aircraftregister_US)

    #Set path to store data
    path = os.path.join(os.path.dirname(os.path.abspath("")), "raw_data", "preproc_data_jonas", "{}.csv".format(int(pflights_df_final.time.unique()[0])))

    # Store preprocessed DataFrame to csv
    pflights_df_final.to_csv(path)

    print('Final DataFrame stored as .csv under {}'.format(path))

Load raw .csv files ...
Start preprocessing ...
Shape before filter_passenger_carriers: (6163, 12)
Shape before get_aircraft_data: (4121, 15)
Shape before merge_seat_data: (4115, 19)
Shape before merge_seat_data_US: (4115, 21)
Shape before impute_missing_seats: (4114, 20)
Shape before increase_capacity_cheap_airlines: (4114, 20)
Shape before rev_geocode: (4114, 20)
Preprocessing finished
Final DataFrame stored as .csv under /home/philipp_heitmann/TravelDashboard/raw_data/preproc_data_jonas/1654167831.csv
Load raw .csv files ...
Start preprocessing ...
Shape before filter_passenger_carriers: (5789, 12)
Shape before get_aircraft_data: (3460, 15)
Shape before merge_seat_data: (3455, 19)
Shape before merge_seat_data_US: (3455, 21)
Shape before impute_missing_seats: (3455, 20)
Shape before increase_capacity_cheap_airlines: (3455, 20)
Shape before rev_geocode: (3455, 20)
Preprocessing finished
Final DataFrame stored as .csv under /home/philipp_heitmann/TravelDashboard/raw_data/preproc_data_j

ValueError: Length mismatch: Expected axis has 1 elements, new values have 18 elements

In [48]:
# test reading stored files
# pd.read_csv("/home/philipp_heitmann/TravelDashboard/raw_data/preproc_data_jonas/1654467161.csv", index_col=0)

In [49]:
# pflights_df_final.shape